In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1,'/home/jovyan/test_surge_models/surgeNN')
import os
from surgeNN.io import load_predictors
import datetime as dt
import gcsfs
import fnmatch
fs = gcsfs.GCSFileSystem() #list stores, stripp zarr from filename, load 

In [7]:
ref_time = dt.datetime(1950,1,1) #1950-01-01 00:00:00

#get the original HadGEM3-GC31-HM calendar
time_ds = xr.open_dataset('gs://'+fnmatch.filter(fs.ls('gs://leap-persistent/timh37/HighResMIP/surgeNN_predictors/'),'*den_helder*')[0],engine='zarr')[['time']].load() #todo: locally stored predictors
time = time_ds.time.values 

hours_since_ref = (time_ds.time-time_ds.time[0]).values/np.timedelta64(1, 'h') + 3 #time0 = 1950-01-01 03:00:00

#loop over timesteps
new_times=[]
for t,this_time in enumerate(time):
    this_yr = this_time.year
    num_days_in_this_yr = pd.Timestamp(this_yr, 12, 31).dayofyear
    
    fraction_of_360yr = hours_since_ref[t]/24 - 360 * (this_yr - ref_time.year) #fraction of 360days
    
    if fraction_of_360yr == 0:
        new_time = dt.datetime(this_yr,1,1)
    elif 0 < fraction_of_360yr < 360:
        new_time = dt.datetime(this_yr,1,1) + dt.timedelta(days=num_days_in_this_yr/360 * fraction_of_360yr) #stretch
    else:
        raise Exception('this should not happen')
    new_times.append(new_time)
    
stretched_time_ds = time_ds.copy(deep=True)
stretched_time_ds['time'] = new_times

#round to 10min (output frequency of GTSM)
how = 'after' #['nearest','before','after']

times = stretched_time_ds.time.values
minutes = stretched_time_ds.time.dt.minute.values
seconds = stretched_time_ds.time.dt.second.values

rounded_times = []
for t,time in enumerate(times):
    time = pd.Timestamp(time)
    discard = dt.timedelta(minutes=int(minutes[t]) % 10,seconds=int(seconds[t]))
    
    rounded_time = time-discard

    if how=='before':
        rounded_times.append(rounded_time)
    elif how=='after':
        if discard > dt.timedelta(minutes=0):
            rounded_times.append(rounded_time + dt.timedelta(minutes=10))
        else:
            rounded_times.append(rounded_time)
    elif how=='nearest':
        if discard >= dt.timedelta(minutes=5):
            rounded_times.append(rounded_time + dt.timedelta(minutes=10))
        else:
            rounded_times.append(rounded_time)
            
out_ds = xr.Dataset(data_vars=dict(a=(['time'],np.zeros(len(rounded_times)))),
                    coords=dict( time=rounded_times ), )
out_ds.to_netcdf('/home/jovyan/test_surge_models/input/stretched_3hourly_360day_HadGEM3_rounded10min_'+how+'.nc',mode='w')

**Deltares code:**

```python
t=maps.getRelativeTimes()
t_reftime=maps.getReferenceTime() #datetime object
for iT, filein_time in enumerate(t): #loop over timesteps in current input file
    var=maps._ds.variables[var_ncvarname][iT,:,:]
    nc_timevar = maps._ds.variables['time']
    if 'hours' in nc_timevar.units:
        filein_time_h = filein_time ###TH: this is simply the nth hour of the whole timeseries
    elif 'days' in nc_timevar.units:
        filein_time_h = filein_time*24
    elif 'minutes' in nc_timevar.units:
        filein_time_h = filein_time/60
    elif 'seconds' in nc_timevar.units:
        filein_time_h = filein_time/3600
    else:
        err_line = 'ERROR: no known time unit (days/hours/minutes/seconds since...)'
        file_err.write('%s\n'%(err_line))
        raise Exception(err_line)
    if (nc_calendar == '360_day') or (nc_calendar == '365_day'): ###TH: hadgem would be 360_day, check ref date?
        if t_reftime.month == 1 and t_reftime.day == 1 and t_reftime.hour == 0 and t_reftime.minute == 0 and t_reftime.second == 0:
            nc_ndayofyear = filein_time_h/24-daysinyear_fixed*(yr-t_reftime.year) ###TH: works out which day of current year (yr?) in original calendar the current timestep is based on the hour
        else: #more complicated method of does not start on 1jan
            filein_time_date = num2date(filein_time,units=nc_timevar.units,calendar=nc_timevar.calendar)
            filein_time_datetime = dt.datetime(filein_time_date.year,filein_time_date.month,filein_time_date.day,filein_time_date.hour,filein_time_date.minute,filein_time_date.second)
            nc_ndayofyear = (filein_time_datetime-dt.datetime(yr,1,1)).total_seconds()/3600/24
            if nc_calendar == '365_day' and calendar.isleap(yr) and filein_time_datetime>=dt.datetime(yr,2,29):
                nc_ndayofyear = nc_ndayofyear-1
        #print(nc_ndayofyear)
        #calculate gregorian date by calculating nday of the year
        if nc_ndayofyear <= 0: #negative values, is spinup period ###TH: this doesn't apply for me?
            datei = dt.datetime(yr,1,1)+dt.timedelta(days=nc_ndayofyear)
        elif 0 < nc_ndayofyear < daysinyear_fixed: #apply stretching for values within current year (excluding 1jan at beginning and end)
            nday_factor = daysinyear/daysinyear_fixed ###TH: these would be days between 0 and 360, compute fraction of year they represent, then multiply with desired days in year (daysinyear? how computed?)
            datei = dt.datetime(yr,1,1)+dt.timedelta(days=nc_ndayofyear*nday_factor)
            #datei = np.datetime64('%i-01-01'%(yr))+np.timedelta64(nc_ndayofyear*nday_factor,'D') #does not work
        else: #take times relative to 1jan next year if in that year (>=360)
            datei = dt.datetime(yr+1,1,1)+dt.timedelta(days=nc_ndayofyear-daysinyear_fixed)
        #calculate new relative time for standard calendar
        fileout_time_h = (datei-t_reftime).total_seconds()/3600
    else:
        datei = num2date(filein_time,units=nc_timevar.units,calendar=nc_timevar.calendar)
        fileout_time_h = filein_time_h
    #print(datei)
    datei_all.append(datei)
    if datei in dates_fileoutreal:
        tid_fileout = dates_fileoutall.index(datei)
        if dates_fileoutall_fillbool[tid_fileout,iR] == False:
            dates_fileoutall_fillbool[tid_fileout,iR] = True
        else:
            err_line = 'ERROR: double value for %s, variable %s'%(dates_fileoutall[tid_fileout],var_standard_name)
            file_err.write('%s\n'%(err_line)) #comment these two lines if you want to deliberately merge overlapping files
            raise Exception(err_line) #comment these two lines if you want to deliberately merge overlapping files
        if iR==0: #fill in time array only for first variable
            dest.setNewRelativeTimes(reftime,[fileout_time_h],t_reftime,tid_fileout)
        #move 180:360 part to -180:0 so field now runs from longitute -180 to 180
        new_var=np.hstack((var[:,part1],var[:,part2]))
        dest._ds.variables[var_ncvarname][tid_fileout,:,:]=new_var
maps.close()
```